# Módulo 4: Control de calidad de datos NGS

## Descripción general

Un proceso típico de secuenciación del genoma completo implica la preparación de muestras, la preparación de bibliotecas y la secuenciación. Los errores que ocurren en cada uno de estos pasos pueden afectar negativamente la calidad de la información de la secuencia. Por ejemplo, las muestras podrían mezclarse durante la preparación de muestras o bibliotecas, o se pueden encontrar errores durante la secuenciación misma. Si estos errores no se eliminan de las lecturas sin procesar, es posible que se incorporen a la salida del análisis y sean más difíciles de resolver más adelante. Por lo tanto, es importante realizar controles de calidad en las lecturas de secuencia sin procesar antes de comenzar su análisis.

Estos son algunos de los parámetros para el control de calidad de datos NGS.

- Compruebe si hay cebadores y adaptadores
- Recorte los extremos de baja calidad
- Eliminar lecturas de baja calidad
- Eliminar secuencias cortas
- Eliminar lecturas con una alta proporción de bases ambiguas ("N")
- Eliminar duplicados


Hay varias herramientas bioinformáticas disponibles para evaluar la calidad de los datos de lectura, aquí discutiremos una de las herramientas más utilizadas, llamada [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) que está disponible gracias al Instituto Babraham.



### Instalar condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Instalar programas

In [ ]:
# Instalar FastQC y trimmomatic
!conda install -c bioconda fastqc -y
!conda install -c bioconda trimmomatic -y

### Descargar los datos

In [ ]:
!wget https://zenodo.org/records/14231070/files/Module_4.tar.gz

### Extraer el archivo .tar.gz

In [ ]:
!tar xvf Module_4.tar.gz

Navegaremos a la carpeta que contiene los archivos pareados FASTQ ERR2667694_1.fastq.gz y ERR2667694_2.fastq.gz los cuales corresponden al run accession ERR2667694 del proyecto [PRJEB3084](https://www.ebi.ac.uk/ena/browser/view/PRJEB3084)	

Algunos datos importantes de la muestra:

- País de origen: Perú
- Organismo: *Streptococcus pneumoniae*
- Instrumento: ILLUMINA
- Modelo del instrumento: HiSeq X Ten
- Conteo de reads: 3418378
- Conteo de bases: 516175078
- Nombre del centro: Wellcome Sanger Institute;SC
- Diseño de librerías:  PAIRED
- Estrategia de librerías: WGS


In [ ]:
# Cambiar de directorio
%cd Module_4/

In [ ]:
# Enumerar o listar todos los archivos
!ls

Ejecutar FastQC

In [ ]:
# Correr fastqc
!fastqc *.fastq.gz

Tendrás la siguiente salida:

![qc](images/fastqc1.png)

La carpeta QC ahora tendrá los siguientes archivos:

![qc](images/fastqc2.png)

Después de verificar todos los parámetros, se generará el informe de QC para las lecturas en los archivos **ERR2667694_1_fastqc.html** y  **ERR2667694_2_fastqc.html**  

Descargue los archivos .html en su ordenador. En la parte izquierda de la pantalla de Colab encontrará un ícono de una carpeta, seleccione y busque los archivos. Seleccione el archivo y seleccione "Descargar": 

![qc](images/fastqc3.png)


**Ahora exploremos el archivo ERR2667694_1_fastqc.html!**

### 01. Estadísticas básicas


El módulo de Estadísticas Básicas genera unas estadísticas de composición sencillas para el archivo analizado.

- **Nombre de archivo**: el nombre de archivo original del archivo que se analizó.

- **Tipo de archivo**: indica si el archivo parecía contener llamadas de base reales o datos de espacio de color que debían convertirse en llamadas de base.

- **Codificación**: indica qué codificación ASCII de valores de calidad se encontró en este archivo

- **Secuencias totales**: recuento del número total de secuencias procesadas. Hay dos valores reportados, reales y estimados. De momento estos siempre serán los mismos. En el futuro, se puede analizar solo un subconjunto de secuencias y estimar el número total para acelerar el análisis, pero como hemos visto, las secuencias problemáticas no están distribuidas uniformemente a través de un archivo, lo hemos desactivado por ahora.

- **Tamaño de la secuencia**: proporciona el tamaño de la secuencia más corta y más larga del conjunto. Si todas las secuencias tienen la misma longitud, solo se informa un valor.

- **%GC**: el %GC general de todas las bases en todas las secuencias

![qc](images/QC1.png)

### 02. Calidad por cada base de la secuencia 

Navegando por el archivo HTML podemos encontrar la cifra de "Calidad por cada base de la secuencia" como se muestra arriba. El gráfico representa la calidad promedio de las bases en toda la longitud de lectura. El eje Y del gráfico muestra las puntuaciones de calidad. El espacio coloreado en las regiones verdes muestra una calidad alta, la siguiente región coloreada en ámbar refleja una calidad aceptable y las regiones en rojo muestran una calidad baja. Por lo tanto, si observa a sus secuencias en la región roja, significa que hay una mayor probabilidad de que un llamado de base sea incorrecto.

Para cada posición se dibuja un gráfico tipo BoxWhisker. Los elementos del gráfico son los siguientes:


- La línea roja central es el valor de la mediana.
- El cuadro amarillo representa el rango intercuartílico (25 - 75)
- Los bigotes superior e inferior representan los 10
- La línea azul representa la calidad media.

La calidad de las llamadas en la mayoría de las plataformas se degradará a medida que avanza la ejecución, por lo que es común ver que la llamada de bases cae en las áreas naranjas hacia el final de una lectura.

Cabe mencionar que hay varias formas diferentes de codificar un puntaje de calidad en un archivo FastQ. FastQC intenta determinar automáticamente qué método de codificación se usó, pero en algunos conjuntos de datos muy limitados es posible que lo adivine incorrectamente (¡irónicamente, solo cuando sus datos son universalmente muy buenos!). El título del gráfico describirá la codificación que FatsQC cree que usó su archivo.

![qc](images/QC2.png)

### 03. Calidad de la secuencia "Per tile"

Este gráfico solo aparecerá en los resultados de su análisis si está utilizando una biblioteca Illumina que conserva sus identificadores de secuencia originales. Codificado en estos está el mosaico de la celda de flujo del que proviene cada lectura. El gráfico le permite ver los puntajes de calidad de cada poso en todas sus bases para ver si hubo una pérdida de calidad asociada con solo una parte de la celda de flujo.

El gráfico muestra la desviación de la calidad promedio de cada poso. Los colores están en una escala de frío a caliente, siendo los colores fríos las posiciones en las que la calidad era igual o superior al promedio de esa base en la tirada, y los colores más cálidos indican que una baldosa tenía peores cualidades que otras baldosas para esa base. Una buena trama debe ser azul por todas partes.

Las razones para ver advertencias o errores en este gráfico pueden ser problemas transitorios, como burbujas que atraviesan la celda de flujo, o pueden ser problemas más permanentes, como manchas en la celda de flujo o desechos dentro del carril de la celda de flujo.

![qc](images/QC3.png)

### 04. Puntuaciones de calidad por secuencia

El informe de puntaje de calidad por secuencia le permite ver si un subconjunto de sus secuencias tiene valores de calidad universalmente bajos. Suele ocurrir que un subconjunto de secuencias tendrá una calidad deficiente en general, a menudo porque la imagen es deficiente (en el borde del campo de visión, etc.); sin embargo, estas deberían representar solo un pequeño porcentaje de las secuencias totales.

Si una proporción significativa de las secuencias en una ejecución tiene una calidad general baja, esto podría indicar algún tipo de problema sistemático, posiblemente con solo una parte de la ejecución (por ejemplo, un extremo de una celda de flujo).

![qc](images/QC4.png)

### 05. Por contenido de secuencia base

Esta métrica sigue la proporción de cada posición de base en un archivo para el que se ha llamado a cada una de las cuatro bases de ADN normales.

En una biblioteca aleatoria, se esperaría que hubiera poca o ninguna diferencia entre las diferentes bases de una corrida, por lo que las líneas en este gráfico deben correr paralelas entre sí. La cantidad relativa de cada base debe reflejar la cantidad total de bases en el genoma, pero en cualquier caso no deben estar muy desequilibradas entre sí.

Vale la pena señalar que algunos tipos de biblioteca siempre producirán una composición de secuencia sesgada, normalmente al comienzo de la lectura. Las bibliotecas producidas mediante primers con hexámeros aleatorios (incluidas casi todas las bibliotecas de RNA-seq) y las que se fragmentaron con transposasas heredan un sesgo intrínseco en las posiciones en las que comienzan las lecturas. Este sesgo no se refiere a una secuencia absoluta, sino que proporciona un enriquecimiento de varios k-mers diferentes en el extremo 5' de las lecturas. Si bien este es un verdadero sesgo técnico, no es algo que pueda corregirse recortando y, en la mayoría de los casos, no parece afectar negativamente el análisis posterior. Sin embargo, producirá una advertencia o un error en este módulo.

![qc](images/QC5.png)

### 06. Contenido de GC por secuencia

Este módulo mide el contenido de GC en toda la longitud de cada secuencia en un archivo y lo compara con una distribución normal modelada del contenido de GC.

En una biblioteca aleatoria normal, se esperaría ver una distribución más o menos normal del contenido de GC donde el pico central corresponde al contenido general de GC del genoma subyacente. Dado que no conocemos el contenido de GC del genoma, el contenido modal de GC se calculó mal a partir de los datos observados y se utilizó para construir una distribución de referencia.

Una distribución de forma inusual podría indicar una biblioteca contaminada o algún otro tipo de subconjunto sesgado. Una distribución normal que se desplaza indica algún sesgo sistemático que es independiente de la posición base. Si hay un sesgo sistemático que crea una distribución normal desplazada, el módulo no lo marcará como un error, ya que no sabe cuál es el contenido de GC de su genoma.

![qc](images/QC6.png)

### 07. Por contenido base N

Si un secuenciador no puede realizar un llamado de base con suficiente confianza, normalmente sustituirá una N en lugar de una llamada de base convencional.

Este módulo traza el porcentaje de llamadas de base en cada posición para la que se llamó una N.

No es inusual ver una proporción muy baja de N que aparecen en una secuencia, especialmente cerca del final de una secuencia. Sin embargo, si esta proporción supera un pequeño porcentaje, sugiere que el pipeline de análisis no pudo interpretar los datos lo suficientemente bien como para realizar llamados de base válidos.

![qc](images/QC7.png)

### 08. Distribución de longitud de secuencia

Algunos secuenciadores de alto rendimiento generan fragmentos de longitud uniforme, pero otros pueden contener lecturas de longitudes muy variables. Incluso con bibliotecas de longitud uniforme, algunas canalizaciones recortarán secuencias para eliminar llamadas de base de baja calidad desde el final.

Este módulo genera un gráfico que muestra la distribución de tamaños de fragmentos en el archivo que se analizó.

En muchos casos, esto producirá un gráfico simple que muestra un pico de un solo tamaño, pero para archivos FastQ de longitud variable, esto mostrará las cantidades relativas de cada tamaño diferente de fragmentos de secuencia.

![qc](images/QC8.png)

### 09. Niveles de duplicación de secuencias

En una biblioteca diversa, la mayoría de las secuencias ocurrirán solo una vez en el conjunto final. Un nivel bajo de duplicación puede indicar un nivel muy alto de cobertura de la secuencia objetivo, pero es más probable que un nivel alto de duplicación indique algún tipo de sesgo de enriquecimiento (p. ej., PCR sobre amplificación).

Este módulo cuenta el grado de duplicación de cada secuencia en una biblioteca y crea un gráfico que muestra el número relativo de secuencias con diferentes grados de duplicación.

Para reducir los requisitos de memoria para este módulo, solo se analizan las secuencias que aparecen primero en las primeras 100000 secuencias de cada archivo, pero esto debería ser suficiente para obtener una buena impresión de los niveles de duplicación en todo el archivo. Cada secuencia se rastrea hasta el final del archivo para dar un recuento representativo del nivel de duplicación general. Para reducir la cantidad de información en el gráfico final, cualquier secuencia con más de 10 duplicados se coloca en contenedores agrupados para dar una impresión clara del nivel de duplicación general sin tener que mostrar cada valor de duplicación individual.

Debido a que la detección de duplicación requiere una coincidencia de secuencia exacta en toda la longitud de la secuencia, cualquier lectura de más de 75 pb de longitud se trunca a 50 pb para los fines de este análisis. Aun así, es más probable que las lecturas más largas contengan errores de secuenciación que aumentarán artificialmente la diversidad observada y tenderán a sub-representar secuencias altamente duplicadas.

El gráfico muestra la proporción de la librería que se compone de secuencias en cada uno de los diferentes niveles de duplicación. Hay dos líneas en el gráfico. La línea azul toma el conjunto de secuencias completos y muestra cómo se distribuyen sus niveles de duplicación. La línea roja, muestra las secuencias dereplicadas y las proporciones que se muestran son las proporciones del conjunto dereplicado que provienen de diferentes niveles de duplicación en los datos originales.

En una librería apropiadamente diversa, la mayoría de las secuencias deberían caer en el extremo izquierdo del gráfico tanto en el rojo como en el azul. Un nivel general de enriquecimiento, que indica una secuencia excesiva en la biblioteca, tenderá a aplanar las líneas, bajando el extremo inferior y, en general, elevando otras categorías. Los enriquecimientos más específicos de subconjuntos o la presencia de contaminantes de baja complejidad tenderán a producir picos hacia la derecha de la gráfica. Estos altos picos de duplicación aparecerán con mayor frecuencia en el seguimiento azul, ya que constituyen una gran proporción de la biblioteca original, pero generalmente desaparecen en el seguimiento de lectura, ya que constituyen una proporción insignificante del conjunto dereplicado. Si los picos persisten en el trazo azul, esto sugiere que hay una gran cantidad de secuencias altamente duplicadas diferentes que podrían indicar un conjunto de contaminantes o una duplicación técnica muy severa.

El módulo también calcula una pérdida general esperada de secuencias si se duplicara la biblioteca. Esta cifra principal se muestra en la parte superior de la gráfica y da una impresión razonable del nivel general potencial de pérdida.

![qc](images/QC9.png)

### 10. Secuencias sobrerrepresentadas

Una librería normal contendrá un conjunto diverso de secuencias, sin que ninguna secuencia individual constituya una pequeña fracción del total. Descubrir que una sola secuencia está muy sobrerrepresentada en el conjunto significa que tiene una gran importancia biológica o indica que la biblioteca está contaminada o que no es tan diversa como esperaba.

Este módulo enumera todas las secuencias que componen más de 0.1.

Para cada secuencia sobrerrepresentada, el programa buscará coincidencias en una base de datos de contaminantes comunes e informará el acierto que encuentre. Los hits deben tener al menos 20 pb de longitud y no tener más de 1 error de coincidencia. Encontrar una coincidencia no significa necesariamente que esta sea la fuente de la contaminación, pero puede indicarle la dirección correcta. También vale la pena señalar que muchas secuencias de adaptadores son muy similares entre sí, por lo que es posible que obtenga un informe que no es técnicamente correcto, pero que tiene una secuencia muy similar a la coincidencia real. 

![qc](images/QC10.png)

### 11. Contenido de Adaptadores

El módulo de contenido k-mer hará un análisis genérico de todos los k-mers en su librería para encontrar aquellos que no tienen cobertura uniforme a lo largo de sus lecturas. Esto puede encontrar varias fuentes diferentes de sesgo en la librería que pueden incluir la presencia de adaptadores al final de sus secuencias.

Sin embargo, puede encontrar que la presencia de cualquier secuencia sobrerrepresentada en su librería (como los dímeros de adaptadores) hará que la gráfica de k-mer esté dominada por los k-mers que contienen estas secuencias, y que no siempre es fácil ver si hay otros sesgos presentes en los que podría estar interesado.

Una clase obvia de secuencias que quizás desee analizar son las secuencias de adaptadores. Es útil saber si su biblioteca contiene una cantidad significativa de adaptadores para poder evaluar si necesita recortar el adaptador o no. Aunque el análisis de k-mer teóricamente puede detectar este tipo de contaminación, no siempre es claro. Por lo tanto, este módulo realiza una búsqueda específica de un conjunto de k-mers definidos por separado y le dará una vista de la proporción total de su librería que contiene estos k-mers. Siempre se generará un seguimiento de resultados para todas las secuencias presentes en el archivo de configuración del adaptador para que pueda ver el contenido de adaptadores de su librería, incluso si es bajo.

El gráfico en sí muestra un conteo porcentual acumulativo de la proporción de su librería en la que se ha visto cada una de las secuencias del adaptador en cada posición. Una vez que se ha visto una secuencia en una lectura, se cuenta como presente hasta el final de la lectura, por lo que los porcentajes que ve solo aumentarán a medida que avanza la lectura.

![qc](images/QC11.png)

Aquí está el resumen de todas las estadísticas según la página del desarrollador:  https://www.bioinformatics.babraham.ac.uk/projects/fastqc/

___

# Recorte de Adaptadores

## Descripción general

El recorte de secuencias de adaptadores a partir de datos de lectura cortos es un paso de preprocesamiento común durante el análisis de datos NGS. Al realizar secuenciación pareada, la superposición entre la lectura directa e inversa se puede utilizar para identificar el exceso de secuencias de adaptadores.

La generación de archivos FASTQ de Illumina incluye una opción de recorte de adaptadores para la eliminación de secuencias de adaptador de los extremos de lectura 3’. Las secuencias del adaptador deben eliminarse de las lecturas porque interfieren con los análisis posteriores, como la alineación de las lecturas con una referencia. Los adaptadores contienen los sitios de unión del primer de secuenciación, las secuencias de índice y los sitios que permiten que los fragmentos de la librería se adhieran al a la celda de flujo. Las librerías preparadas con los kits de preparación de librerías de Illumina requieren que se recorten los adaptadores solo en los extremos de 3’, ya que las secuencias de adaptadores no se encuentran en los extremos de 5’.

Lecturas adicionales

1. https://support.illumina.com/bulletins/2016/04/adapter-trimming-why-are-adapter-sequences-trimmed-from-only-the--ends-of-reads.html 

2. https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-1069-7


Evaluaremos la calidad de **spneumo_R1.fastq.gz** y **spneumo_R2.fastq.gz** en la carpeta *Adapter_trimming*

In [ ]:
# Cambiar de directorio a Adapter_trimming
%cd Adapter_trimming

In [ ]:
# Correr fastqc
!fastqc spneumo_R1.fastq.gz spneumo_R2.fastq.gz

Descargue los archivos HTML que se encuentra en el panel izquierdo y ábralo en su navegador. Verá la calidad de las lecturas.

![example](images/example.png))

La herramienta trimmomatic se utiliza para eliminar adaptadores, recortar lecturas de baja calidad y eliminar secuencias cortas.

Para ejecutar la herramienta trimmomatic ejecutaremos el comando:

In [ ]:
!trimmomatic PE spneumo_R1.fastq.gz spneumo_R2.fastq.gz spneumo_R1.trimmed.fastq.gz  /dev/null spneumo_R2.trimmed.fastq.gz /dev/null ILLUMINACLIP:./adapters/TruSeq3-PE.fa:2:30:10 SLIDINGWINDOW:4:20 MINLEN:36

Una explicación de este comando es la siguiente:


- **trimmomatic**: es la herramienta

- **PE**: Que tomará el archivo final emparejado como entrada

- **spneumo_R1.fastq.gz**: El primer nombre de archivo de entrada

- **spneumo_R2.fastq.gz**: el segundo nombre de archivo de entrada

- **spneumo_R1.trimmed.fastq.gz**: el archivo de salida para los pares supervivientes del archivo _1

- **/dev/null**: descarta el archivo de salida para lecturas huérfanas del archivo _1

- **pneumo_R1.trimmed.fastq.gz**: el archivo de salida para los pares supervivientes del archivo _2

- **/dev/null**: descarta el archivo de salida para lecturas huérfanas del archivo _2

- **ILLUMINACLIP**: adapters/TruSeq3-PE.fa:2:30:10: para recortar los adaptadores illumina del archivo de entrada utilizando las secuencias de adaptador enumeradas en TruSeq3-PE.fa. Los números 2:30:10 le indican a trimmomatic cómo manejar las coincidencias de secuencia con los adaptadores TruSeq3.

- **SLIDINGWINDOW:4:20**: Para usar una ventana deslizante de tamaño 4 que eliminará bases si su puntuación de phred es inferior a 20

- **MINLEN:36**: Esto descartará y leerá que no quedan al menos 36 bases después de este paso de recorte

Tendrás el siguiente resultado:

![trimming](images/trimming.png)

Ejecute nuevamente fastqc sobre los archivos recortados o trimados ".trimmed.fastq.gz" para verificar la calidad de las lecturas 

In [ ]:
# correr fastqc
!fastqc *.trimmed.fastq.gz

### Preguntas


> ¿Qué porcentaje de lecturas descartó de su muestra?

> ¿Qué porcentaje de lecturas mantuviste?

> ¿En qué se diferencia el informe HTML de spneumo_R1.trimmed.fastq.gz/spneumo_R2.trimmed.fastq.gz  del de spneumo_R1.fastq.gz/spneumo_R2.fastq.gz?

## BONO!

Si estás trabajando con BASH en su computadora o en un HPC y tiene demasiados archivos, puedes optimizar los comandos, los bucles (loops) son muy útiles para grandes conjuntos de datos.

Aquí hay una manera de hacerlo.

Crea un nuevo script bash usando nano llamado `trimming.sh`

In [ ]:
# No ejecute esta celda
# Crea un script para ejecutar el recorte
!nano trimming.sh

Luego copia y pega el siguiente scrip en su nuevo archivo:

In [ ]:
#!/bin/bash
#Author: Nathalia Portilla

for i in $(ls *_1.trimmed.fastq.gz); do

NAME=$(basename $i _1.trimmed.fastq.gz)
echo "$NAME"
j="${NAME}_1.trimmed.fastq.gz"
echo "$j"
k="${NAME}_2.trimmed.fastq.gz"
echo "$k"

trimmomatic PE $j $k ${NAME}_1.trimmed.fastq.gz /dev/null ${NAME}_2.trimmed.fastq.gz /dev/null ILLUMINACLIP:TruSeq3-PE.fa:2:30:10 SLIDINGWINDOW:6:30 MINLEN:50

done

Guarda el archivo.

Finalmente, puedes ejecutarlo:

In [ ]:
#No ejecute esta celda
!bash trimming.sh

*Adaptado de:*

- Advanced Bioinformatics Course developed for the GPS and JUNO projects - Wellcome Sanger Insitute
    
*Modificado por Luisa Sacristán (Universidad de los Andes-CABANA)*